A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

The Head Data Scientist at Training Data Ltd. has asked you to create a DataFrame called ds_jobs_clean that stores the data in customer_train.csv much more efficiently. Specifically, they have set the following requirements:

- Columns containing integers must be stored as 32-bit integers (int32).
- Columns containing floats must be stored as 16-bit floats (float16).
- Columns containing nominal categorical data must be stored as the category data type.
- Columns containing ordinal categorical data must be stored as ordered categories, with an order that reflects the natural order of the column.
- The columns of ds_jobs_clean must be in the same order as the original dataset.
- The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.
- The final line must be ds_jobs_clean.head(100) to return only the first 100 rows.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

# Exploratory data analysis

In [1]:
# Loading 'customer_train.csv'
import pandas as pd
ds_jobs = pd.read_csv('customer_train.csv')
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [2]:
# Determining the type and contents of each columns
ds_jobs.info()
ds_jobs.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

student_id                19158
city                        123
city_development_index       93
gender                        3
relevant_experience           2
enrolled_university           3
education_level               5
major_discipline              6
experience                   22
company_size                  8
company_type                  6
last_new_job                  6
training_hours              241
job_change                    2
dtype: int64

In [3]:
# Displaying the unique values of the each column
for colx in ds_jobs.select_dtypes(include = ['object']):
        print(f'The unique values of the column "{colx}" are:')
        print(f'{ds_jobs[colx].value_counts()}\n')

The unique values of the column "city" are:
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64

The unique values of the column "gender" are:
Male      13221
Female     1238
Other       191
Name: gender, dtype: int64

The unique values of the column "relevant_experience" are:
Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64

The unique values of the column "enrolled_university" are:
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64

The unique values of the column "education_level" are:
Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64

The unique values of the column "major_discipline" a

# Converting data types 

In [4]:
# Convert columns containing integers to the int32 type, floats to the float16 type, and nominal categories to the category type
ds_jobs_clean = pd.read_csv('customer_train.csv')
for coln in ds_jobs_clean.columns:
    if ds_jobs_clean[coln].dtype == 'int64':
        ds_jobs_clean[coln] = ds_jobs_clean[coln].astype('int32') 
    elif ds_jobs_clean[coln].dtype == 'float64' and ds_jobs_clean[coln].nunique()>2:
        ds_jobs_clean[coln] = ds_jobs_clean[coln].astype('float16') 
    else:
        ds_jobs_clean[coln] = ds_jobs_clean[coln].astype('category')
print(ds_jobs_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  category
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  category
dtypes: category(11), float

# Converting ordered categories

In [5]:
# Convert columns containing ordinal categorical data into ordered categories.
col_ord_categ = ["relevant_experience", "enrolled_university", "education_level", 
                 "experience", "company_size", "last_new_job"]
for col_cat in col_ord_categ:
    print(f'The list of categories for column "{col_cat}" are:')  
    print(f'{ds_jobs_clean[col_cat].cat.categories}\n')

The list of categories for column "relevant_experience" are:
Index(['Has relevant experience', 'No relevant experience'], dtype='object')

The list of categories for column "enrolled_university" are:
Index(['Full time course', 'Part time course', 'no_enrollment'], dtype='object')

The list of categories for column "education_level" are:
Index(['Graduate', 'High School', 'Masters', 'Phd', 'Primary School'], dtype='object')

The list of categories for column "experience" are:
Index(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2',
       '20', '3', '4', '5', '6', '7', '8', '9', '<1', '>20'],
      dtype='object')

The list of categories for column "company_size" are:
Index(['10-49', '100-499', '1000-4999', '10000+', '50-99', '500-999',
       '5000-9999', '<10'],
      dtype='object')

The list of categories for column "last_new_job" are:
Index(['1', '2', '3', '4', '>4', 'never'], dtype='object')



In [6]:
ds_jobs_clean['relevant_experience'] = ds_jobs_clean['relevant_experience'].cat.reorder_categories(
    new_categories = ['No relevant experience', 'Has relevant experience'], ordered = True)
ds_jobs_clean['relevant_experience'].cat.categories                              

Index(['No relevant experience', 'Has relevant experience'], dtype='object')

In [7]:
ds_jobs_clean['enrolled_university'] = ds_jobs_clean['enrolled_university'].cat.reorder_categories(
    new_categories = ['no_enrollment', 'Part time course', 'Full time course'], ordered =True)
ds_jobs_clean['enrolled_university'].cat.categories  

Index(['no_enrollment', 'Part time course', 'Full time course'], dtype='object')

In [8]:
ds_jobs_clean['education_level'] = ds_jobs_clean['education_level'].cat.reorder_categories(
    new_categories = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered =True)
ds_jobs_clean['education_level'].cat.categories  

Index(['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], dtype='object')

In [9]:
ds_jobs_clean['experience'] = ds_jobs_clean['experience'].cat.reorder_categories(
    new_categories = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9','10', '11',
                      '12', '13', '14', '15', '16', '17', '18', '19','20','>20'], ordered =True)
ds_jobs_clean['experience'].cat.categories 

Index(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '>20'],
      dtype='object')

In [10]:
ds_jobs_clean['company_size'] = ds_jobs_clean['company_size'].cat.reorder_categories(
    new_categories = ['<10', '10-49', '50-99','100-499', '500-999','1000-4999', '5000-9999','10000+'], ordered =True)
ds_jobs_clean['company_size'].cat.categories 

Index(['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999',
       '10000+'],
      dtype='object')

In [11]:
ds_jobs_clean['last_new_job'] = ds_jobs_clean['last_new_job'].cat.reorder_categories(
    new_categories = ['never', '1', '2', '3', '4', '>4'], ordered =True)
ds_jobs_clean['last_new_job'].cat.categories 

Index(['never', '1', '2', '3', '4', '>4'], dtype='object')

# Reordering columns

In [12]:
# Reorder the columns of ds_jobs_clean so it has the same column ordering as the original dataset.
ds_jobs_clean = ds_jobs_clean[ds_jobs.columns]
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  category
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  category
dtypes: category(11), float

# Filtering on ordered categorical columns

In [13]:
# Filter the DataFrame to only contain students with 10 or more years of experience at companies with at least 1000 employees.
ds_jobs_clean = ds_jobs_clean[ds_jobs_clean['experience'].isin(list(ds_jobs_clean['experience'].cat.categories)[10:]) 
                              &
                              ds_jobs_clean['company_size'].isin(list(ds_jobs_clean['company_size'].cat.categories)[-3:])]
ds_jobs_clean.head(100)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0.0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0.0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0.0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0.0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,28453,city_16,0.910156,Male,Has relevant experience,no_enrollment,Masters,STEM,10,10000+,Pvt Ltd,1,18,0.0
931,6304,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,1000-4999,Public Sector,>4,142,0.0
933,31368,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,10,1000-4999,Pvt Ltd,1,314,0.0
959,21276,city_75,0.938965,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,4,94,0.0
